## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_m2det import models
from keras_m2det.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_m2det.utils.visualization import draw_box, draw_caption
from keras_m2det.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Load M2det model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {15: 'cat', 33:'kitte'} #16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

Loading the model from  ../snapshots/resnet50_coco_best_v2.1.0.h5 ...
Instructions for updating:
Colocations handled automatically by placer.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Run detection on example

In [7]:
from skimage.io import imshow, imread, imsave


path_to_images_folder = '/Users/user/Desktop/aylifind/detection/cats_original/'


def make_detection(i, image_path):
    
    image = imread(image_path)
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)
        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        return b
    

for i, image_path in enumerate(os.listdir(path_to_images_folder)):
    if image_path.endswith('.jpg'):
        try:
            print(image_path)
            box = make_detection(i, path_to_images_folder + image_path)
            print(box)
            x1 = box[0]
            y1 = box[1]
            x2 = box[2]
            y2 = box[3]
            image = imread(path_to_images_folder + image_path)
            image_resized = image[y1:y2, x1:x2]
            imsave('/Users/user/Desktop/aylifind/detection/m2det_e/' + 'image_{}.jpg'.format(i), image_resized)
        except:
            continue

63.jpg
processing time:  8.644181966781616
[  7 253 621 925]
162.jpg
processing time:  9.684443950653076
[  0  28 649 362]
412.jpg
processing time:  8.669713973999023
[ 458  195 1035  737]
214.jpg
processing time:  8.045487880706787
201.jpg
processing time:  9.489276885986328
[207 122 567 798]
215.jpg
processing time:  8.512682914733887
[576 245 963 689]
413.jpg
processing time:  8.769375085830688
[ 551  323 1256  708]
361.jpg
processing time:  8.362387895584106
[311 140 490 426]
163.jpg
processing time:  7.741447925567627
[ 13 107 550 881]
62.jpg
processing time:  10.142225980758667
[173 339 569 707]
74.jpg
processing time:  8.647829055786133
[ 65 220 770 800]
161.jpg
processing time:  8.396111011505127
[  4  47 700 628]
411.jpg
processing time:  9.093616247177124
[240 226 554 748]
405.jpg
processing time:  9.0364830493927
[119  73 241 294]
363.jpg
processing time:  8.994427919387817
[116 288 664 960]
216.jpg
processing time:  9.933983087539673
[190 251 549 645]
202.jpg
processing tim

processing time:  12.275305032730103
391.jpg
processing time:  7.39528226852417
[176   0 913 822]
92.jpg
processing time:  10.217226028442383
[ 47 332 658 809]
193.jpg
processing time:  9.428054094314575
[ 48  38 561 793]
191.jpg
processing time:  8.91947889328003
[ 62  37 723 547]
152.jpg
processing time:  10.176330089569092
[168  72 355 310]
84.jpg
processing time:  9.002824068069458
[   0   66 1270  955]
393.jpg
processing time:  10.52029299736023
[157 108 643 788]
422.jpg
processing time:  11.729712963104248
[  5   7 520 967]
344.jpg
processing time:  9.644298076629639
[318 455 473 664]
231.jpg
processing time:  8.533108234405518
[ 125  244  653 1073]
351.jpg
processing time:  7.797131061553955
[  0 176 714 789]
423.jpg
processing time:  8.645780324935913
[  5   7 756 528]
392.jpg
processing time:  6.130764961242676
[ 88  82 834 884]
91.jpg
processing time:  7.733378171920776
[585  71 718 307]
153.jpg
processing time:  8.72286605834961
[  4  98 507 294]
184.jpg
processing time:  8.

In [ ]:
!pwd

In [ ]:
plt.figure(figsize=(9, 7))
img_path = '/Users/user/Desktop/aylifind/detection/cats_original/162.jpg'
img = imread(img_path)

imshow(img)



In [ ]:
y1 = 28
y2 = 362
x1 = 0
x2 = 649
img_resized = img[y1:y2, x1:x2]
imshow(img_resized)